In [1]:
import lightgbm as lgbm
from scipy import sparse as ssp
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
path = "data/"

train = pd.read_csv(path+'application_train.csv')
train_label = train['TARGET']
train_id = train['SK_ID_CURR']
test = pd.read_csv(path+'application_test.csv')
test_id = test['SK_ID_CURR']

column_type = pd.read_csv('data/application_columns.csv')

train_len = len(train_id)
test_len = len(test_id)

In [3]:
y_train = train['TARGET'].values

drop_feature = ['SK_ID_CURR','TARGET']

train_test = pd.concat([train,test],axis=0)

print(train.shape)
print(test.shape)
print(train_test.shape)
print(train.shape[0] + test.shape[0] == train_test.shape[0])

(307511, 122)
(48744, 121)
(356255, 122)
True


In [4]:
# cat_features = [f for f in feature_names if X[f].dtype == 'object']
# num_features = [c for c in feature_names if ('CNT_'in c or 'AMT_'in c)]
# flg_features = [c for c in feature_names if ('FLAG_' in c or )]


cat_features = column_type.loc[column_type.categorical==1].column.values.tolist()
num_features = column_type.loc[column_type.numerical==1].column.values.tolist()
flg_features = column_type.loc[column_type.flag==1].column.values.tolist()

#application_features = application_cat_cols + application_num_cols + application_flag_cols

In [5]:
# train['missing'] = (train==-1).sum(axis=1).astype(float)
# test['missing'] = (test==-1).sum(axis=1).astype(float)
# num_features.append('missing')

In [6]:
from sklearn.preprocessing import Imputer
#from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute

train_test[cat_features] = train_test[cat_features].fillna('missing') 
train_test['missing_cols'] =  train_test.isnull().sum(axis=1)

num_features.append('missing_cols')

#imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)

impute_train_only = False
imp = Imputer(missing_values='NaN', strategy='median', axis=0)

if impute_train_only:
    imp.fit(train[num_features])
    train_test[num_features] = imp.transform(train_test[num_features])
else:
    train_test[num_features] = imp.fit_transform(train_test[num_features])

    

In [7]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
# Encode each category into a integer
train_test[cat_features] = train_test[cat_features].apply(preprocessing.LabelEncoder().fit_transform)


In [8]:

one_hot = False

if one_hot:
    train_test = pd.get_dummies(train_test,columns=cat_features)
    cat_features = []



train = train_test.loc[train_test.TARGET.notnull()]

test = train_test.loc[train_test.TARGET.isnull()]

test = test.drop(['TARGET'],axis=1, inplace = False)

print(train.shape[0] == train_len)
print(test.shape[0] == test_len)

True
True


In [9]:

X_train = train.drop(['SK_ID_CURR', 'TARGET'], axis = 1)
y_train = train['TARGET']

X_test = test.drop(['SK_ID_CURR'], axis = 1)


print(X_train.shape[1] == X_test.shape[1])
print(y_train.shape[0] == train_len)
print(X_test.shape[0] == test_len)
print(X_test.shape[1])




True
True
True
121


In [10]:
###### the main NN script##########3

from keras.layers import Dense, Dropout, Embedding, Flatten, Input, merge
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from time import time
import datetime
from keras.models import Model
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
from itertools import combinations
from scipy import sparse
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.preprocessing import LabelEncoder


/home/qile/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
def interaction_features(train, test, fea1, fea2, prefix):
    train['inter_{}*'.format(prefix)] = train[fea1] * train[fea2]
    train['inter_{}/'.format(prefix)] = train[fea1] / train[fea2]

    test['inter_{}*'.format(prefix)] = test[fea1] * test[fea2]
    test['inter_{}/'.format(prefix)] = test[fea1] / test[fea2]

    return train, test

def proj_num_on_cat(train_df, test_df, target_column, group_column):
    """
    :param train_df: train data frame
    :param test_df:  test data frame
    :param target_column: name of numerical feature
    :param group_column: name of categorical feature
    """
    train_df['row_id'] = range(train_df.shape[0])
    test_df['row_id'] = range(test_df.shape[0])
    train_df['train'] = 1
    test_df['train'] = 0
    all_df = train_df[['row_id', 'train', target_column, group_column]].append(test_df[['row_id','train',
                                                                                        target_column, group_column]])
    grouped = all_df[[target_column, group_column]].groupby(group_column)
    the_size = pd.DataFrame(grouped.size()).reset_index()
    the_size.columns = [group_column, '%s_size' % target_column]
    the_mean = pd.DataFrame(grouped.mean()).reset_index()
    the_mean.columns = [group_column, '%s_mean' % target_column]
    the_std = pd.DataFrame(grouped.std()).reset_index().fillna(0)
    the_std.columns = [group_column, '%s_std' % target_column]
    the_median = pd.DataFrame(grouped.median()).reset_index()
    the_median.columns = [group_column, '%s_median' % target_column]
    the_stats = pd.merge(the_size, the_mean)
    the_stats = pd.merge(the_stats, the_std)
    the_stats = pd.merge(the_stats, the_median)

    the_max = pd.DataFrame(grouped.max()).reset_index()
    the_max.columns = [group_column, '%s_max' % target_column]
    the_min = pd.DataFrame(grouped.min()).reset_index()
    the_min.columns = [group_column, '%s_min' % target_column]

    the_stats = pd.merge(the_stats, the_max)
    the_stats = pd.merge(the_stats, the_min)

    all_df = pd.merge(all_df, the_stats, how='left')

    selected_train = all_df[all_df['train'] == 1]
    selected_test = all_df[all_df['train'] == 0]
    selected_train.sort_values('row_id', inplace=True)
    selected_test.sort_values('row_id', inplace=True)
    selected_train.drop([target_column, group_column, 'row_id', 'train'], axis=1, inplace=True)
    selected_test.drop([target_column, group_column, 'row_id', 'train'], axis=1, inplace=True)

    selected_train, selected_test = np.array(selected_train), np.array(selected_test)
    print(selected_train.shape, selected_test.shape)
    return selected_train, selected_test

In [12]:
num_features = num_features
feature_names = list(X_train.columns.values)


In [13]:
list(X_train.columns.values)

['AMT_ANNUITY',
 'AMT_CREDIT',
 'AMT_GOODS_PRICE',
 'AMT_INCOME_TOTAL',
 'AMT_REQ_CREDIT_BUREAU_DAY',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_CREDIT_BUREAU_QRT',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'AMT_REQ_CREDIT_BUREAU_YEAR',
 'APARTMENTS_AVG',
 'APARTMENTS_MEDI',
 'APARTMENTS_MODE',
 'BASEMENTAREA_AVG',
 'BASEMENTAREA_MEDI',
 'BASEMENTAREA_MODE',
 'CNT_CHILDREN',
 'CNT_FAM_MEMBERS',
 'CODE_GENDER',
 'COMMONAREA_AVG',
 'COMMONAREA_MEDI',
 'COMMONAREA_MODE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_ID_PUBLISH',
 'DAYS_LAST_PHONE_CHANGE',
 'DAYS_REGISTRATION',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'ELEVATORS_AVG',
 'ELEVATORS_MEDI',
 'ELEVATORS_MODE',
 'EMERGENCYSTATE_MODE',
 'ENTRANCES_AVG',
 'ENTRANCES_MEDI',
 'ENTRANCES_MODE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'FLAG_CONT_MOBILE',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT

In [16]:
X_test.head(2)

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,TOTALAREA_MODE,WALLSMATERIAL_MODE,WEEKDAY_APPR_PROCESS_START,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_AVG,YEARS_BUILD_MEDI,YEARS_BUILD_MODE,missing_cols
0,20560.5,568800.0,450000.0,135000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0392,5,5,0.9732,0.9732,0.9732,0.7552,0.7585,0.7648,26.0
1,17370.0,222768.0,180000.0,99000.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0690,7,0,0.9816,0.9816,0.9816,0.7552,0.7585,0.7648,45.0


In [17]:
train_cat = X_train[cat_features]
train_num = X_train[num_features]
test_cat = X_test[cat_features]
test_num = X_test[num_features]

max_cat_values = list(train_test[cat_features].max())


train_list = train_num
test_list = test_num

all_data = np.vstack([train_num.values, test_num.values])
scaler = StandardScaler()
scaler.fit(all_data)
X = scaler.transform(train_num.values)
X_test = scaler.transform(test_num.values)
print(X.shape, X_test.shape)

(307511, 72) (48744, 72)


In [21]:
max_cat_values

[1, 2, 1, 1, 7, 7, 4, 5, 5, 18, 6, 23, 57, 4, 3, 7, 2]

In [143]:
cv_train = np.zeros(len(y_train))
cv_pred = np.zeros(len(test_id))

In [79]:
X_cat = train_cat.as_matrix()
X_test_cat = test_cat.as_matrix()

x_test_cat = []
for i in range(X_test_cat.shape[1]):
    x_test_cat.append(X_test_cat[:, i].reshape(-1, 1))
x_test_cat.append(X_test)

In [80]:
X_cat.shape

(307511, 17)

In [89]:
len(x_test_cat)

18

In [123]:
def nn_model():
    inputs_cat = []
    flatten_layers_cat = []
    inputs_other = []
    flatten_layers_other = []
    for e, c in enumerate(cat_features):
        input_c = Input(shape=(1, ), dtype='int32')
        num_c = max_cat_values[e] + 1 ## correct?
        embed_c = Embedding(
            num_c,
            6,
            input_length=1#,
            #name= 'embedding_layer_' + str(e)
        )(input_c)
        embed_c = Dropout(0.25)(embed_c)
        flatten_c = Flatten()(embed_c)

        inputs_cat.append(input_c)
        flatten_layers_cat.append(flatten_c)
    
    flatten_layers_cat = merge(flatten_layers_cat, mode='concat', name='embedding')
    
    
    
#     input_num = Input(shape=(X.shape[1],), dtype='float32')
#     flatten_layers.append(input_num)
#     inputs.append(input_num)
    
    
    input_num = Input(shape=(X.shape[1],), dtype='float32')
    flatten_layers_other.append(input_num)
    
    inputs_cat.append(input_num)
    
    #flatten_layers_other = merge(flatten_layers_other, mode='concat', name='other')
    
    flatten_layers_all = merge([flatten_layers_cat,input_num], mode='concat', name='all')
    
    fc1 = Dense(512, init='he_normal')(flatten_layers_all)
    fc1 = PReLU()(fc1)
    fc1 = BatchNormalization()(fc1)
    fc1 = Dropout(0.75)(fc1)

    fc1 = Dense(64, init='he_normal')(fc1)
    fc1 = PReLU()(fc1)
    fc1 = BatchNormalization()(fc1)
    fc1 = Dropout(0.5)(fc1)

    outputs = Dense(1, init='he_normal', activation='sigmoid')(fc1)

    model = Model(input = inputs_cat, output = outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return (model)


In [144]:
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve

# X_cat = train_cat.as_matrix()
# X_test_cat = test_cat.as_matrix()
# cv_train = np.zeros(len(y_train))
# cv_pred = np.zeros(len(test_id))


# x_test_cat = []
# for i in range(X_test_cat.shape[1]):
#     x_test_cat.append(X_test_cat[:, i].reshape(-1, 1))
# x_test_cat.append(X_test)

begintime = time()

NFOLDS = 5
kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=218)

for (inTr, inTe) in kfold.split(X, y_train):
    xtr = X[inTr]
    ytr = y_train[inTr]
    xte = X[inTe]
    yte = y_train[inTe]

    xtr_cat = X_cat[inTr]
    xte_cat = X_cat[inTe]

    # get xtr xte cat
    xtr_cat_list, xte_cat_list = [], []
    for i in range(xtr_cat.shape[1]):
        xtr_cat_list.append(xtr_cat[:, i].reshape(-1, 1))
        xte_cat_list.append(xte_cat[:, i].reshape(-1, 1))

    xtr_cat_list.append(xtr)
    xte_cat_list.append(xte)

    model = nn_model()
#     def get_rank(x):
#         return pd.Series(x).rank(pct=True).values
    model.fit(xtr_cat_list, ytr, epochs=4, batch_size=512, verbose=2, validation_data=[xte_cat_list, yte])
    #cv_train[inTe] += get_rank(model.predict(x=xte_cat_list, batch_size=512, verbose=0)[:, 0])
    cv_train[inTe] += model.predict(x=xte_cat_list, batch_size=512, verbose=0)[:, 0]

    print(roc_auc_score(train_label[inTe], cv_train[inTe]))
    #cv_pred += get_rank(model.predict(x=x_test_cat, batch_size=512, verbose=0)[:, 0])
    cv_pred += model.predict(x=x_test_cat, batch_size=512, verbose=0)[:, 0]
#print(s)
#print(roc_auc_score(train_label, cv_train / (1. * (s + 1))))
print(roc_auc_score(train_label, cv_train))

print(str(datetime.timedelta(seconds=time() - begintime)))
# if save_cv:
# pd.DataFrame({'id': test_id, 'target': get_rank(cv_pred * 1./ (NFOLDS * num_seeds))}).to_csv('../model/keras5_pred.csv', index=False)
# pd.DataFrame({'id': train_id, 'target': get_rank(cv_train * 1. / num_seeds)}).to_csv('../model/keras5_cv.csv', index=False)


/home/qile/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/qile/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/home/qile/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/qile/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(512, kernel_initializer="he_normal")`
/home/qile/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: UserWarning: Update your `D

Train on 246008 samples, validate on 61503 samples
Epoch 1/4
 - 11s - loss: 0.4094 - val_loss: 0.2686
Epoch 2/4
 - 6s - loss: 0.2789 - val_loss: 0.2612
Epoch 3/4
 - 6s - loss: 0.2685 - val_loss: 0.2554
Epoch 4/4
 - 6s - loss: 0.2631 - val_loss: 0.2547
0.7375765506588142
Train on 246009 samples, validate on 61502 samples
Epoch 1/4
 - 11s - loss: 0.4088 - val_loss: 0.2701
Epoch 2/4
 - 6s - loss: 0.2778 - val_loss: 0.2600
Epoch 3/4
 - 6s - loss: 0.2686 - val_loss: 0.2552
Epoch 4/4
 - 6s - loss: 0.2629 - val_loss: 0.2545
0.7380705424734021
Train on 246009 samples, validate on 61502 samples
Epoch 1/4
 - 12s - loss: 0.4096 - val_loss: 0.2697
Epoch 2/4
 - 6s - loss: 0.2776 - val_loss: 0.2635
Epoch 3/4
 - 6s - loss: 0.2682 - val_loss: 0.2567
Epoch 4/4
 - 6s - loss: 0.2633 - val_loss: 0.2543
0.739280508601511
Train on 246009 samples, validate on 61502 samples
Epoch 1/4
 - 12s - loss: 0.3997 - val_loss: 0.2711
Epoch 2/4
 - 6s - loss: 0.2763 - val_loss: 0.2604
Epoch 3/4
 - 6s - loss: 0.2670 - val

In [125]:
cv_train.shape

(307511,)

In [145]:
print(roc_auc_score(train_label, cv_train))


0.738402161457933


In [146]:
print(roc_auc_score(train_label, cv_train))


0.738402161457933


In [142]:
cv_train.max()

2.211038887500763

In [147]:
save_cv= True 
if save_cv:
    pd.DataFrame({'SK_ID_CURR': test_id, 'TARGET': cv_pred * 1./ (NFOLDS)}).to_csv('model/keras5_pred.csv', index=False)
    pd.DataFrame({'SK_ID_CURR': train_id, 'TARGET': cv_train}).to_csv('model/keras5_cv.csv', index=False)
    

In [148]:
!kg submit 'model/keras5_pred.csv' -u username -p password -c home-credit-default-risk -m "FIRST NN 4 epho 0.7398976445912158"


0.729


In [149]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_614 (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
input_615 (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
input_616 (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
input_617 (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
input_618 

In [101]:
inputs = []
flatten_layers = []
for e, c in enumerate(cat_features):
    name= 'embedding_layer_' + str(e)
    inputs.append(name)



In [102]:
inputs

['embedding_layer_0',
 'embedding_layer_1',
 'embedding_layer_2',
 'embedding_layer_3',
 'embedding_layer_4',
 'embedding_layer_5',
 'embedding_layer_6',
 'embedding_layer_7',
 'embedding_layer_8',
 'embedding_layer_9',
 'embedding_layer_10',
 'embedding_layer_11',
 'embedding_layer_12',
 'embedding_layer_13',
 'embedding_layer_14',
 'embedding_layer_15',
 'embedding_layer_16']

In [150]:
model.get_layer('embedding').output

<tf.Tensor 'embedding_494/concat:0' shape=(?, ?) dtype=float32>

In [168]:
embedded_features_model = Model(inputs=model.input[0:-1], outputs=model.get_layer('embedding').output) 


In [167]:
model.input[0:-1]

[<tf.Tensor 'input_614:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'input_615:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'input_616:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'input_617:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'input_618:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'input_619:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'input_620:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'input_621:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'input_622:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'input_623:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'input_624:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'input_625:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'input_626:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'input_627:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'input_628:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'input_629:0' shape=(?, 1) dtype=int32>,
 <tf.Tensor 'input_630:0' shape=(?, 1) dtype=int32>]

In [165]:
xtr_cat_list[17].shape

(246009, 72)

In [169]:
embedded_features_train = embedded_features_model.predict(xtr_cat_list[0:-1])

In [176]:
embedded_features_train.shape

(246009, 102)

In [99]:
len(xtr_cat_list)

18

In [171]:
embedded_features_train.shape

(246009, 102)

In [172]:
embedded_features_train[0]

array([ 0.1796346 ,  0.18139878, -0.16695075, -0.2034567 ,  0.14897706,
       -0.14072777, -0.12307332,  0.04396079, -0.11771409, -0.20023814,
        0.10661795, -0.21523684,  0.16264682,  0.13940103, -0.19735573,
        0.11535189, -0.10254943, -0.14167127,  0.02101806,  0.17427869,
        0.0276799 , -0.10359795,  0.1223225 , -0.1040542 , -0.14784789,
       -0.06371597,  0.11416012,  0.03310515,  0.03340962, -0.14494094,
        0.12431963,  0.09730543,  0.08371974,  0.1484717 ,  0.10437058,
       -0.11263569,  0.13712513, -0.13281515, -0.06487958,  0.1373709 ,
       -0.05361715, -0.08410319, -0.03786104, -0.07568193, -0.05253077,
        0.01883043, -0.0107798 , -0.0018675 , -0.13564554,  0.1061696 ,
        0.06811448, -0.12573121, -0.17051165,  0.12894347, -0.05158365,
       -0.09940769, -0.05015821, -0.07494926,  0.02835752, -0.07486555,
       -0.00689175,  0.0739475 ,  0.00406729,  0.00509849, -0.06369077,
       -0.04260473, -0.05918357,  0.09956083,  0.00392976,  0.02

In [68]:
X_train.shape

(307511, 121)

In [74]:
embedded_features_train[:,1].shape

(246009,)

0.8000006503832383

In [77]:
X_test_cat.shape

(48744, 17)

In [78]:
X_test_cat.shape[1]

17

In [87]:
len(xtr_cat_list)

18

In [91]:
X.shape

(307511, 72)